In [2]:

from PIL import Image, ImageDraw,ImageFont
import torch
import torchvision.models as models
from torchvision.transforms import functional as F
from yolov5.models.experimental import attempt_load
from yolov5.utils.general import non_max_suppression



In [3]:
!nvidia-smi
import os
HOME = os.getcwd()
print(HOME)

Fri Sep  8 05:30:20 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 531.41                 Driver Version: 531.41       CUDA Version: 12.1     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                      TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf            Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce GTX 1650 Ti    WDDM | 00000000:01:00.0 Off |                  N/A |
| N/A   38C    P8                3W /  N/A|      0MiB /  4096MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [22]:
from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.0.134  Python-3.11.3 torch-2.0.1+cpu CPU
Setup complete  (12 CPUs, 23.4 GB RAM, 328.1/709.4 GB disk)


In [4]:
from ultralytics import YOLO



In [5]:
model1 = YOLO('./runs/detect/train2/weights/best.pt')
model2 = YOLO('./runs/detect/train2/weights/last.pt')

In [6]:
model1.names
model2.names

{0: 'Ink-stamps'}

In [7]:
img_path = './POD1.PNG'
img = Image.open(img_path).convert('RGB')
myFont = ImageFont.truetype('./Roboto-Bold.ttf', 30)

In [8]:
results = model1.predict(source=img_path, conf=0.90)
results2 = model2.predict(source=img_path, conf=0.30)


image 1/1 d:\3.projects_by_tech\Python\STAMP_SEARCH_TEST\NewTestData\POD1.PNG: 352x800 4 Ink-stampss, 526.1ms
Speed: 8.0ms preprocess, 526.1ms inference, 8.0ms postprocess per image at shape (1, 3, 352, 800)

image 1/1 d:\3.projects_by_tech\Python\STAMP_SEARCH_TEST\NewTestData\POD1.PNG: 352x800 2 Ink-stampss, 327.4ms
Speed: 8.4ms preprocess, 327.4ms inference, 0.0ms postprocess per image at shape (1, 3, 352, 800)


In [10]:
combined_results = results+results2
combined_results.sort(key=lambda x: x[4], reverse=True)
combined_results

IndexError: index 4 is out of bounds for dimension 0 with size 4

In [101]:
i=0

for det in results[0].boxes.xyxy:
    x1, y1, x2, y2 = det
    x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
    #print(x1, y1, x2, y2, conf, cls)

    # Draw bounding box on the original image
    draw = ImageDraw.Draw(img)
    draw.rectangle([x1, y1, x2, y2], outline="red", width=3)
    draw2 = ImageDraw.Draw(img)
    draw2.text((x1+8, y1+3),"r1: "+str(int(results[0].boxes.conf[i]*100))+"%", font=myFont,fill=(255, 0, 0),outline=(0,0,0))
    #draw.text((x1, y1), str(int(results[0].boxes.conf[i])),"Marilyn Monroe",fill=(0, 0, 0))
    i=i+1



In [29]:
i=0
for det in results2[0].boxes.xyxy:
    x1, y1, x2, y2 = det
    x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
    #print(x1, y1, x2, y2, conf, cls)

    # Draw bounding box on the original image
    draw = ImageDraw.Draw(img)
    draw.rectangle([x1, y1, x2, y2], outline="red", width=3)
    draw2 = ImageDraw.Draw(img)
    draw2.text((x1+28, y1+50),"r2: "+str(int(results2[0].boxes.conf[i]*100))+"%", font=myFont,fill=(255, 0, 0),outline=(0,0,0))
    #draw.text((x1, y1), str(int(results[0].boxes.conf[i])),"Marilyn Monroe",fill=(0, 0, 0))
    i=i+1

img.show()

In [36]:
seal_present="false"
if(results2[0].boxes.xyxy.size()[0]>=2):
    seal_present="true"
else:
    seal_present="false"

In [3]:
import easyocr
reader = easyocr.Reader(['en'])
result = reader.readtext('./POD1.PNG', detail = 0)
print(type(result))

Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


<class 'list'>


In [38]:
ref_no=""
for txts in result:
    if(txts.__contains__("Ref#")):
        ref_no=txts.split("#")[1].split("/")[0]

ref_no

'852957008'

In [42]:
import xlwt
from xlwt import Workbook
  
# Workbook is created
wb = Workbook()
  
# add_sheet is used to create sheet.
sheet1 = wb.add_sheet('Sheet 1')

sheet1.write(0,0,"Reference no.")
sheet1.write(0,1,"File Path")
sheet1.write(0,2,"Two Seal Present")
sheet1.write(1,0,int(ref_no))
sheet1.write(1,1,img_path)

sheet1.write(1,2,seal_present)

  
wb.save('scan_output.xlsx')